In [ ]:
from keras.layers.pooling.global_max_pooling2d import GlobalMaxPool2D
from sklearn.svm import SVC

# Your existing model definition
def getModel(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    transfer = keras.applications.MobileNetV3Large(
        weights='imagenet', include_top=False, input_tensor=model_input
    )
    x = transfer.get_layer(index=142).output
    x = SpatialDropout2D(0.15)(x)
    x = Conv2D(filters=64, kernel_size=1, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-1))(x)
    x = GlobalMaxPool2D()(x)
    x = Dropout(0.15)(x)
    model_output = Dense(1, activation='sigmoid')(x)
    return keras.Model(inputs=model_input, outputs=model_output)

# Train your deep learning model
model = getModel(image_size=target, num_classes=1)
# ... (train the model using your training data)

# Freeze the deep learning model
for layer in model.layers:
    layer.trainable = False

# Extract features from the deep learning model
feature_extractor = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
train_features = feature_extractor.predict(train_data)
val_features = feature_extractor.predict(val_data)

# Train the SVM
svm = SVC()
svm.fit(train_features, train_labels)

# Evaluate the SVM
accuracy = svm.score(val_features, val_labels)
print(f"SVM accuracy: {accuracy:.4f}")